In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install a bunch of packages

In [2]:
!pip install flask
!pip install flask-ngrok
!pip install pyngrok
!pip install wtforms
!pip install flask_wtf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.5/152.5 kB 5.0 MB/s eta 0:00:00


# Import necessary stuff

In [1]:
import getpass
import os
import threading
from pyngrok import ngrok, conf
import socket

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
from PIL import Image
import cv2
from keras.models import load_model
from flask import Flask, request, render_template
from werkzeug.utils import secure_filename
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [3]:
print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken")
conf.get_default().auth_token = '<your-authtoken>'

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken


# Check whether the model is uploaded properly

In [19]:
path_model = '/content/drive/MyDrive/MyGitProject_B/model.keras'
model =load_model(path_model)
# print('Model loaded. Check http://127.0.0.1:5000/')

labels = {0: 'Healthy', 1: 'Powdery', 2: 'Rust'}


def getResult(image_path):
    img = load_img(image_path, target_size=(225,225))
    x = img_to_array(img)
    x = x.astype('float32') / 255.
    x = np.expand_dims(x, axis=0)
    predictions = model.predict(x)[0]
    return predictions

In [20]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rashikrahmanpritom/plant-disease-recognition-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'plant-disease-recognition-dataset' dataset.
Path to dataset files: /kaggle/input/plant-disease-recognition-dataset


In [21]:
#Double-check whether or not the model was loaded properly

from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

def preprocess_image(image_path, target_size=(225, 225)):
    img = load_img(image_path, target_size=target_size)
    x = img_to_array(img)
    x = x.astype('float32') / 255.
    x = np.expand_dims(x, axis=0)
    return x

In [22]:
path_x = '/content/drive/MyDrive/Kaggle_API/8ddaa5a5caa5caa8.jpg'
x = preprocess_image(path_x)

In [23]:
predictions = model.predict(x)
predictions[0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 830ms/step


array([0.91356665, 0.01016542, 0.07626793], dtype=float32)

In [24]:
path_x = '/content/drive/MyDrive/Kaggle_API/80bc7d353e163e85.jpg'
x = preprocess_image(path_x)
predictions = model.predict(x)
predictions[0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


array([0.17800695, 0.79919195, 0.02280114], dtype=float32)

# The Application


In [4]:
from flask_ngrok import run_with_ngrok

In [5]:
app = Flask(__name__, static_folder='/content/drive/MyDrive/MyGitProject_B/static',
            template_folder='/content/drive/MyDrive/MyGitProject_B/templates')
# run_with_ngrok(app)

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(5000).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}/\"".format(public_url, 5000))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

path_model = '/content/drive/MyDrive/MyGitProject_B/model.keras'
model =load_model(path_model)
# print('Model loaded. Check http://127.0.0.1:5000/')

labels = {0: 'Healthy', 1: 'Powdery', 2: 'Rust'}


def getResult(image_path):
    img = load_img(image_path, target_size=(225,225))
    x = img_to_array(img)
    x = x.astype('float32') / 255.
    x = np.expand_dims(x, axis=0)
    predictions = model.predict(x)[0]
    return predictions


@app.route('/', methods=['GET'])
def index():
    return render_template('index.html')


@app.route('/predict', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        f = request.files['file']

        basepath = '/content/drive/MyDrive/MyGitProject_B'

        folder_path = os.path.join(basepath, 'uploads')
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        file_path = os.path.join(
            basepath, 'uploads', secure_filename(f.filename))
        f.save(file_path)

        predictions = getResult(file_path)
        predicted_label = labels[np.argmax(predictions)]

        return str(predicted_label)
    return

# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()

 * ngrok tunnel "https://656c2045f1a8.ngrok-free.app" -> "http://127.0.0.1:5000/"


 * Debug mode: off
